In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms


In [6]:
ds=CIFAR10(root="data",download=True,transform=transforms.ToTensor())

Files already downloaded and verified


In [15]:
mean=[]
std=[]
for x in range(3):
    mean.append(i[x].mean())
    std.append(i[x].std())
print(mean)
print(std)

[tensor(0.5537), tensor(0.4122), tensor(0.2511)]
[tensor(0.1595), tensor(0.1665), tensor(0.1603)]


In [17]:
trans={
    "train":transforms.Compose([
        transforms.Resize(32),
        transforms.RandomCrop(30,padding=True,padding_mode="reflect"),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(180),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    "val":transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [22]:
device="cuda" if torch.cuda.is_available() else "cpu"
batch_size=128

In [21]:
ds=CIFAR10(root="data",transform=trans["train"],train=True)
ts=ds=CIFAR10(root="data",transform=trans["val"],train=False)
dl=DataLoader(ds,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=3)
tl=DataLoader(ts,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=3)

In [52]:
def convBlock(in_channel,out_channel,pool):
    layers=[nn.Conv2d(in_channel,out_channel,kernel_size=3,padding=1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
            ]
    if pool:layers.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*layers)

In [116]:
class Resnet(nn.Module):
    def __init__(self,in_channel,num_classes):
        super().__init__()
        self.conv1=convBlock(in_channel,12,pool=True)
        self.conv2=convBlock(12,24,pool=True)
        self.res1=nn.Sequential(convBlock(24,24,pool=False),convBlock(24,24,pool=False))
        self.conv3=convBlock(24,48,pool=True)
        self.conv4=convBlock(48,124,pool=True)
        self.res2=nn.Sequential(convBlock(124,124,pool=False),convBlock(124,124,pool=False))
        self.classify=nn.Sequential(nn.Flatten(),nn.Linear(124*2*2,num_classes))
    
    def forward(self,x):
        out=self.conv1(x)
        out=self.conv2(out)
        out=self.res1(out)+out
        out=self.conv3(out)
        out=self.conv4(out)
        out=self.res2(out)+out
        out=self.classify(out)
        return out

In [58]:
for i in dl:
    dta=i
    break

In [59]:
dta[0].shape

torch.Size([128, 3, 32, 32])

In [117]:
model=Resnet(in_channel=3,num_classes=10)

In [119]:
dt=torch.unsqueeze(dta[0][0],dim=0)
model(dt)

tensor([[ 0.4061,  0.0772, -0.0696,  2.1243, -0.5149, -0.1483, -0.0998,  2.6446,
         -0.2732, -0.3534]], grad_fn=<AddmmBackward>)

In [125]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [129]:
def fit(epochs,max_lr,model,loss,wd,grad_clip,tl):
    model.to(device)
    optim=torch.optim.Adam(model.parameters(),weight_decay=wd,lr=max_lr)
    sch=torch.optim.lr_scheduler.OneCycleLR(optim,max_lr=max_lr,epochs=epochs,steps_per_epoch=len(tl))
    for epoch in range(epochs):
        model.train()
        for i,j in tl:
            i=i.to(device)
            j=j.to(device)
            pred=model(i)
            ls=loss(pred,j)
            ls.backward()
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(),grad_clip)
            optim.step()
            optim.zero_grad()
            sch.step()
            acc=accuracy(pred,j)
        print(f"loss={ls} acc={acc}")
        

In [130]:
model=Resnet(in_channel=3,num_classes=10)
loss=nn.CrossEntropyLoss()
wDecay=0.01
gradclip=1
maxlr=0.0001
epochs=20



In [131]:
fit(epochs,maxlr,model,loss,wDecay,gradclip,dl)

loss=2.293839693069458 acc=0.1875
loss=1.902808666229248 acc=0.4375
loss=2.0383846759796143 acc=0.3125
loss=1.9003742933273315 acc=0.25
loss=1.8170111179351807 acc=0.375
loss=1.5147364139556885 acc=0.4375
loss=0.8258908987045288 acc=0.8125
loss=0.8549389839172363 acc=0.6875
loss=1.1511389017105103 acc=0.4375
loss=1.019219160079956 acc=0.6875
loss=1.0341283082962036 acc=0.625
loss=0.589480459690094 acc=0.9375
loss=0.8795186281204224 acc=0.6875
loss=0.9090612530708313 acc=0.5625
loss=0.9567352533340454 acc=0.8125
loss=0.8263826370239258 acc=0.75
loss=0.6835426688194275 acc=0.8125
loss=1.1141713857650757 acc=0.5
loss=0.7557905912399292 acc=0.75
loss=0.4566907584667206 acc=0.875


In [133]:
def evaluate(epochs,model,loss,tl):
    for epoch in range(epochs):
        model.eval()
        for i,j in tl:
            i=i.to(device)
            j=j.to(device)
            pred=model(i)
            ls=loss(pred,j)
            acc=accuracy(pred,j)
        print(f"loss={ls} acc={acc}")
        

In [ ]:
evaluate(epochs,model,loss,tl)